In [ ]:
!pip install langchain
!pip install langchain_community
!pip install langchain_chroma
!pip install langchainhub



INDEXING:LOAD

LOADING THE CONTENTS FROM POST TO A TEXT FORMAT

In [ ]:
import bs4
from langchain_community.document_loaders import WebBaseLoader

bs4_strainer = bs4.SoupStrainer('div')
loader=WebBaseLoader(
    web_paths=("https://en.wikipedia.org/wiki/Studio_Ghibli",),
    bs_kwargs={"parse_only": bs4_strainer},
)

docs= loader.load()

len(docs[0].page_content)

print(docs[0].page_content[:500])

INDEXING:SPLIT

SPLITTING DOCUMENT INTO CHUNKS TO FIT INTO CONTEXT WINDOW OF MODELS

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000, chunk_overlap=200, add_start_index=True
)
all_splits = text_splitter.split_documents(docs)
print(type(all_splits))
print(all_splits)

INDEXING:store

STORING DATA INTO CHROMA VECTOR STORE AND OPENAI EMBEDDINGS MODEL

In [ ]:
import os 
from dotenv import load_dotenv

load_dotenv()

In [ ]:
from langchain_chroma import Chroma
from langchain_community.embeddings import HuggingFaceInferenceAPIEmbeddings
##api_key = os.getenv('HF_API_KEY')

embeddings = HuggingFaceInferenceAPIEmbeddings(    
            api_key=os.getenv('HF_API_KEY'), 
            model_name="thenlper/gte-large"
 )
vectorstore = Chroma.from_documents(documents=all_splits, embedding=embeddings)


RETRIEVAL AND GENERATION: RETRIEVAL

In [ ]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

retrieved_docs = retriever.invoke("What are the approaches to Task Decomposition?")

len(retrieved_docs)

print(retrieved_docs[0].page_content)

RETRIEVAL AND GENERATION: GENERATION

In [ ]:
from langchain import hub
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser

import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAI
import os 
from dotenv import load_dotenv

load_dotenv()
api_key = os.getenv('GCP_API_KEY')

llm = GoogleGenerativeAI(model="models/text-bison-001", google_api_key=api_key)

prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("grave of the fireflies")